In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from pathlib import Path

In [12]:
dir_path = "results_with_swap/"
dfs = []
for elem in Path(dir_path).iterdir():
    if elem.suffix == ".csv" and "fasttodd" in elem.name:
        dfs.append(pd.read_csv(elem))

In [13]:
def opt_analysis(before_tcounts:list, after_tcounts:list):
    initial_tcount = 1
    length = len(before_tcounts)
    assert length == len(after_tcounts)
    reduction_rates = []
    avg_before_t, avg_after_t = 0, 0
    for i in range(length):
        avg_before_t += before_tcounts[i]
        avg_after_t += after_tcounts[i]
        reduction_rates.append(
            100 * (before_tcounts[i] - after_tcounts[i]) / (before_tcounts[i] - initial_tcount)
        )
    avg_before_t /= length
    avg_after_t /= length
    return avg_before_t, avg_after_t, np.mean(reduction_rates)


def generate_performance_sheet(dir_path, nqubits:int):
    for elem in Path(dir_path).iterdir():
        if f"n={nqubits}" in elem.name and elem.suffix == ".csv":
            if "fasttodd" in elem.name:
                df_todd = pd.read_csv(elem)
            else:
                df = pd.read_csv(elem)

    avg_before_t_tket, avg_after_t_tket, p_tket = opt_analysis(
        df["before_T_count"], df["tket_after_T_count"]
    )

    avg_before_t_pyzx, avg_after_t_pyzx, p_pyzx = opt_analysis(
        df["before_T_count"], df["pyzx_after_T_count"]
    )

    avg_before_t_tmerge, avg_after_t_tmerge, p_tmerge = opt_analysis(
        df["before_T_count"], df["tmerge_after_T_count"]
    )
    # fasttodd
    avg_before_t_todd, avg_after_t_todd, p_todd = opt_analysis(
        df["before_T_count"][:30], df_todd["fasttodd_after_T_count"]
    )
    avg_ancilla = np.mean(df_todd["fasttodd_ancilla_count"].to_list())
    return pd.DataFrame(
        {
            f"{nqubits}": [
                avg_before_t_tket,
                avg_after_t_tket,
                p_tket,
                avg_before_t_pyzx,
                avg_after_t_pyzx,
                p_pyzx,
                avg_before_t_tmerge,
                avg_after_t_tmerge,
                p_tmerge,
                avg_before_t_todd,
                avg_after_t_todd,
                p_todd,
                avg_ancilla,
            ]
        }
    )

In [17]:
dfs = []
for n in range(2,10):
    dfs.append(
        generate_performance_sheet("results_with_swap/", n)
    )  # "results_with_swap/" or "results/"

In [18]:
df = pd.concat(dfs, axis=1).T
df.columns = [
    "avg_before_tket",
    "avg_after_tket",
    "p_tket",
    "avg_before_pyzx",
    "avg_after_pyzx",
    "p_pyzx",
    "avg_before_tmerge",
    "avg_after_tmerge",
    "p_tmerge",
    "avg_before_fasttodd",
    "avg_after_fasttodd",
    "p_fasttodd",
    "avg_ancilla_fasttodd"
]

In [19]:
df

,avg_before_tket,avg_after_tket,p_tket,avg_before_pyzx,avg_after_pyzx,p_pyzx,avg_before_tmerge,avg_after_tmerge,p_tmerge,avg_before_fasttodd,avg_after_fasttodd,p_fasttodd,avg_ancilla_fasttodd
2,46.68,43.30,7.395257,46.68,32.58,30.863636,46.68,32.58,30.863636,46.666667,34.466667,26.646904,19.866667
3,106.52,104.54,1.878084,106.52,96.44,9.552684,106.52,96.44,9.552684,106.666667,94.666667,11.364296,58.000000
4,190.58,189.60,0.516573,190.58,185.78,2.531019,190.58,185.78,2.531019,190.400000,185.066667,2.815992,116.433333
5,298.24,297.88,0.121305,298.24,295.94,0.773265,298.24,295.94,0.773265,298.133333,273.933333,8.138113,189.300000
6,430.32,430.26,0.013953,430.32,429.84,0.111824,430.32,429.84,0.111824,430.466667,389.066667,9.653519,278.233333
7,586.26,586.10,0.027304,586.26,585.78,0.081958,586.26,585.78,0.081958,586.200000,526.633333,10.169635,382.966667
8,766.10,766.10,0.000000,766.10,766.00,0.013069,766.10,766.00,0.013069,766.200000,466.000000,39.236380,501.100000
9,970.24,970.20,0.004124,970.24,970.08,0.016495,970.24,970.08,0.016495,970.333333,469.833333,51.633852,628.400000
